In [40]:
import pandas as pd

emp = pd.read_excel("employee_data.xlsx")
display(emp.head())
display(emp.tail())

,EmployeeID,Name,Department,Date,Status,TasksCompleted,HoursWorked
0,1,Employee_1,Finance,2025-01-02,Present,3.0,9.3
1,1,Employee_1,Finance,2025-01-03,Present,2.0,8.4
2,1,Employee_1,Finance,2025-01-04,Present,7.0,3.7
3,1,Employee_1,Finance,2025-01-05,Absent,0.0,NaN
4,1,Employee_1,Finance,2025-01-06,Absent,NaN,0.0


,EmployeeID,Name,Department,Date,Status,TasksCompleted,HoursWorked
2095,50,Employee_50,Finance,2025-02-22,Absent,0.0,0.0
2096,50,Employee_50,Finance,2025-02-23,Present,4.0,6.7
2097,50,Employee_50,Finance,2025-02-24,Present,5.0,5.0
2098,50,Employee_50,Finance,2025-02-27,Absent,NaN,NaN
2099,50,Employee_50,Finance,2025-02-28,Present,NaN,7.9


In [41]:
print("Missing Values in Xl File:", emp.isnull().sum())

Missing Values in Xl File: EmployeeID          0
Name                0
Department          0
Date                0
Status            118
TasksCompleted    327
HoursWorked       317
dtype: int64


In [42]:
emp["TasksCompleted"] = emp["TasksCompleted"].fillna(0)
emp["HoursWorked"] = emp["HoursWorked"].fillna(0)
emp["Status"] = emp["Status"].fillna("Absent")


In [43]:
print("After handling the missing values:\n",emp.isnull().sum())

After handling the missing values:
 EmployeeID        0
Name              0
Department        0
Date              0
Status            0
TasksCompleted    0
HoursWorked       0
dtype: int64


In [44]:
emp["Date"] = pd.to_datetime(emp["Date"])
print(emp["Date"].head())
print("\n",emp["Date"].tail())

0   2025-01-02
1   2025-01-03
2   2025-01-04
3   2025-01-05
4   2025-01-06
Name: Date, dtype: datetime64[ns]

 2095   2025-02-22
2096   2025-02-23
2097   2025-02-24
2098   2025-02-27
2099   2025-02-28
Name: Date, dtype: datetime64[ns]


In [22]:
attendance_summary = emp.groupby(["EmployeeID", "Name"])["Date"].nunique().reset_index()
attendance_summary.rename(columns={"Date": "TotalWorkingDays"}, inplace=True)

status_summary = emp.groupby(["EmployeeID", "Name", "Status"])["Date"].count().unstack(fill_value=0).reset_index()

attendance_report = pd.merge(attendance_summary, status_summary, on=["EmployeeID", "Name"])

display(attendance_report.head())


,EmployeeID,Name,TotalWorkingDays,Absent,Present,Work From Home
0,1,Employee_1,42,14,23,5
1,2,Employee_2,42,12,26,4
2,3,Employee_3,42,11,27,4
3,4,Employee_4,42,11,26,5
4,5,Employee_5,42,16,24,2


In [29]:
dept_hours = emp.groupby("Department")["HoursWorked"].mean().reset_index()

top_employees = emp.groupby(["EmployeeID", "Name"])["TasksCompleted"].sum().reset_index()
top_employees = top_employees.sort_values(by="TasksCompleted", ascending=False).head(5)

dept_tasks = emp.groupby("Department")["TasksCompleted"].mean().reset_index()

display("Finding the average hours worked per department:",dept_hours)
display("Identify the top 5 employees with the highest total tasks completed:",top_employees)
display("For each department the average tasks completed per day:",dept_tasks)


'Finding the average hours worked per department:'

,Department,HoursWorked
0,Finance,4.493590
1,HR,4.318155
2,IT,4.429167
3,Marketing,4.442208
4,Sales,3.935000


'Identify the top 5 employees with the highest total tasks completed:'

,EmployeeID,Name,TasksCompleted
29,30,Employee_30,151.0
31,32,Employee_32,149.0
40,41,Employee_41,149.0
49,50,Employee_50,138.0
36,37,Employee_37,135.0


'For each department the average tasks completed per day:'

,Department,TasksCompleted
0,Finance,2.716117
1,HR,2.678571
2,IT,2.741071
3,Marketing,2.571429
4,Sales,2.707143


In [31]:
emp["Month"] = emp["Date"].dt.to_period("M")


monthly_tasks = emp.groupby("Month")["TasksCompleted"].sum().reset_index()
monthly_hours = emp.groupby("Month")["HoursWorked"].mean().reset_index()


monthly_summary = pd.merge(monthly_tasks, monthly_hours, on="Month")
monthly_summary.rename(columns={
    "TasksCompleted": "TotalTasksCompleted",
    "HoursWorked": "AvgHoursWorked"
}, inplace=True)

display(monthly_summary)


,Month,TotalTasksCompleted,AvgHoursWorked
0,2025-01,3041.0,4.327727
1,2025-02,2588.0,4.337100


In [39]:
with pd.ExcelWriter("employee_report.xlsx", engine="openpyxl") as writer:
    emp.to_excel(writer, sheet_name="Manipulated Data", index=False)
    attendance_report.to_excel(writer, sheet_name="Attendance Summary", index=False)
    dept_hours.to_excel(writer, sheet_name="Avg Dept Hours", index=False)
    top_employees.to_excel(writer, sheet_name="Top 5 Employees", index=False)
    dept_tasks.to_excel(writer, sheet_name="Avg Dept Tasks", index=False)
    monthly_summary.to_excel(writer, sheet_name="Monthly Summary", index=False)
